In [1]:
import numba
from numba import njit, prange
import numpy as np
from itertools import repeat
from multiprocessing import Pool, cpu_count
import textwrap
import os
import time
from ctypes import cdll
import ctypes
from ctypes import *



# Реализовывался Toom-3

In [11]:
K = 3

lib = ctypes.CDLL("./libfoo.so")

# lib.add.argtypes = [ctypes.POINTER(c_char), ctypes.POINTER(c_char)]
# lib.add.restype = ctypes.POINTER(c_char)
# lib.subtract.argtypes = [ctypes.c_char_p, ctypes.c_char_p]
# lib.subtract.restype = ctypes.c_char_p
# lib.division.argtypes = [ctypes.c_char_p, ctypes.c_int]
# lib.division.restype = ctypes.c_char_p

lib.pyadd.argtypes = [ctypes.POINTER(
    c_char), ctypes.POINTER(c_char), ctypes.POINTER(c_char)]
lib.pyadd.restype = None
lib.pysubtract.argtypes = [ctypes.POINTER(
    c_char), ctypes.POINTER(c_char), ctypes.POINTER(c_char)]
lib.pysubtract.restype=None
lib.pydivision.argtypes = [ctypes.POINTER(
    c_char), ctypes.POINTER(c_char), ctypes.POINTER(c_char)]
lib.pydivision.restype = None

# c_add = ctypes.CFUNCTYPE(None, ctypes.POINTER(c_char), ctypes.POINTER(
#     c_char), ctypes.POINTER(c_char), ctypes.POINTER(c_char))(lib.pyadd)
# c_sub = ctypes.CFUNCTYPE(None, ctypes.POINTER(c_char), ctypes.POINTER(
#     c_char), ctypes.POINTER(c_char), ctypes.POINTER(c_char))(lib.pysubtract)
# c_div = ctypes.CFUNCTYPE(None, ctypes.POINTER(c_char), ctypes.POINTER(
#     c_char), ctypes.POINTER(c_char), ctypes.POINTER(c_char))(lib.pydivision)

matrix = [[("1", "1"), ("0", "1"), ("0", "1"), ("0", "1"), ("0", "1")],
          [("1", "2"), ("1", "3"), ("-1", "1"), ("1", "6"), ("-2", "1")],
          [("-1", "1"), ("1", "2"), ("1", "2"), ("0", "1"), ("-1", "1")],
          [("-1", "2"), ("1", "6"), ("1", "2"), ("-1", "6"), ("2", "1")],
          [("0", "1"), ("0", "1"), ("0", "1"), ("0", "1"), ("1", "1")]]

# @numba.jit(forceobj=True)
# def sfdsdfadd(lhs, rhs):
#     lhs = np.array([*lhs])
#     rhs = np.array([*rhs])
#     res = np.zeros(lhs.size + rhs.size)

#     lhs = lhs.ctypes.data_as(ctypes.POINTER(ctypes.c_char))
#     rhs = rhs.ctypes.data_as(ctypes.POINTER(ctypes.c_char))
#     res = res.ctypes.data_as(ctypes.POINTER(ctypes.c_char))
    
#     c_add(lhs, rhs, res)

#     return res

@numba.jit(nopython=True)
def eval_i(m, n):
    return max(((len(m) // 4) // 3), ((len(n) // 4) // 3)) + 1


@numba.jit(nopython=True)
def divide(m, n):
    return str(int(m) // int(n))



# @numba.jit(forceobj=True)
# def subtract(lhs, rhs, lib=lib):
#     lhs = ctypes.c_char_p(lhs.encode('utf-8'))
#     rhs = ctypes.c_char_p(rhs.encode('utf-8'))

#     c_output_str = lib.subtract(lhs, rhs)

#     # return ctypes.string_at(c_output_str).decode('utf-8')
#     return c_output_str

# @numba.jit(forceobj=True)
# def division(lhs, rhs, lib=lib):
#     lhs = ctypes.c_char_p(lhs.encode('utf-8'))
#     rhs = ctypes.c_int(rhs)

#     c_output_str = lib.division(lhs, rhs)

#     return ctypes.string_at(c_output_str).decode('utf-8')



@numba.jit(nopython=True)
def multiply_strings(num1, num2):
    if num1 == '0' or num2 == '0':
        return '0'

    result = [0] * (len(num1) + len(num2))

    num1 = num1[::-1]
    num2 = num2[::-1]

    for i in range(len(num1)):
        for j in range(len(num2)):
            digit1 = ord(num1[i]) - ord("0")
            digit2 = ord(num2[j]) - ord("0")
            product = digit1 * digit2

            result[i + j] += product % 10
            result[i + j + 1] += product // 10

            if result[i + j] >= 10:
                result[i + j + 1] += result[i + j] // 10
                result[i + j] %= 10

    while len(result) > 1 and result[-1] == 0:
        result.pop()

    return ''.join(map(str, result[::-1]))

In [16]:
lhs = "11666"
rhs = "123132"

lhs = np.array([*lhs])
rhs = np.array([*rhs])
res = np.array(["0"] * (lhs.size + rhs.size))

lhs_ptr = lhs.ctypes.data_as(ctypes.POINTER(ctypes.c_char))
rhs_ptr = rhs.ctypes.data_as(ctypes.POINTER(ctypes.c_char))
res_ptr = res.ctypes.data_as(ctypes.POINTER(ctypes.c_char))

lib.pyadd(lhs_ptr, rhs_ptr, res_ptr)
res


array(['2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], dtype='<U1')

In [61]:
# @njit
# def row_multiplication(row, col):
#     results_private = ["0", "1"]

#     for i in range(len(col)):
#         tmp = (toom_cook(
#             col[i], row[i][0]), row[i][1])
#         num = str(int(toom_cook(
#             tmp[0], results_private[1])) + int(toom_cook(tmp[1], results_private[0])))
#         denom = toom_cook(tmp[1], results_private[1])

#         results_private[0] = num
#         results_private[1] = denom

#     # return str(int(results_private[0]) //
#     #            int(results_private[1]))

#     return results_private


# @njit(parallel=True)
# def matmul(matrix, vector_):
#     result = ["0"] * len(matrix)
#     for row in prange(len(matrix)):
#         result[row] = row_multiplication(matrix[row], vector_)

#     return result

@numba.jit(forceobj=True)
def toom_cook(m_, n_):
    m = m_
    n = n_

    if (len(m) <= 9 and len(n) <= 9) or (m[0] == '-' and n[0] == '-' and len(m) <= 10 and len(n) <= 10):
        return multiply_strings(m, n)

    m_minus = False
    n_minus = False

    if m[0] == '-':
        m = m[1:]
        m_minus = True

    if n[0] == '-':
        n = n[1:]
        n_minus = True


    B = 4 * eval_i(m, n)

    p = []
    q = []

    for i in range(0, len(m), B):
        p += [m[::-1][i:i+B][::-1]]

    for i in range(0, len(n), B):
        q += [n[::-1][i:i+B][::-1]]


    p = p + ["0"] * (3 - len(p))
    q = q + ["0"] * (3 - len(q))

    p_p = [p[0], add(p[0], add(p[1], p[2])), add(subtract(p[0], p[1]), p[2]),
           add(subtract(p[0], toom_cook("2", p[1])), toom_cook("4", p[2])), p[2]]
    q_q = [q[0], add(q[0], add(q[1], q[2])), add(subtract(q[0], q[1]), q[2]),
           add(subtract(q[0], toom_cook("2", q[1])), toom_cook("4", q[2])), q[2]]
    

    r_in_points = [toom_cook(p_p[0], q_q[0]), toom_cook(p_p[1], q_q[1]), toom_cook(
        p_p[2], q_q[2]), toom_cook(p_p[3], q_q[3]), toom_cook(p_p[4], q_q[4])]

    # res = matmul(matrix, r_in_points)

    return m

    # recompos = [res[0], res[1] + "0" * B, res[2] + "0" *
    #             B * 2, res[3] + "0" * B * 3, res[4] + "0" * B * 4]

    # if (m_minus and n_minus) or (not m_minus and not n_minus):
    #     return str(int(recompos[4]) + int(recompos[3]) + int(recompos[2]) + int(recompos[1]) + int(recompos[0]))

    # return "-" + str(int(recompos[4]) + int(recompos[3]) + int(recompos[2]) + int(recompos[1]) + int(recompos[0]))


In [260]:
np.array([*"123"]).data